<h1><center>Laboratorio 9: Benchmark Estadístico con Reddit 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Sebastián Aguirre
- Nombre de alumno 2: Cristóbal Gallardo


### **Link de repositorio de GitHub:** `https://github.com/cristobalgallardo/MDS7202`

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 3/12/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Utilizar la API de Reddit a partir de la librería `praw` y visualizar cuales post son más probables que sean puntuados positivamente.
- Utilizar `cache_lru` para cachear resultados de funciones.
- Aplicar un atajo estadístico para obtener la mean posterior de datos.
- Medir el tiempo de ejecución como también el uso de memoria de la función anterior.
- Optimizar la función anterior a través de `numba`.

#Importamos librerias utiles 😸

In [ ]:
!pip install "ipython>=7"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%%capture
!pip install praw
!pip install line_profiler
!pip install memory_profiler
!pip install numba

%load_ext autoreload
%autoreload 2

import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://preview.redd.it/3sxusxcjge791.png?auto=webp&s=88bc1f9a3c59eafda24b0e32dd87d7cf596d205d">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema estadísticos, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis [1 Punto]

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a cada liena de código para ser ejecutada. Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [ ] Estudiar la función propuesta por el equipo docente.
- [ ] Estudiar los tiempos de ejecución del código a través de un perfilador.
- [ ] Estudiar la memoria ocupada por el código a través de un perfilador.
- [ ] Comente los resultados. ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?

In [ ]:
%%file bench_module.py
import praw
import numpy as np

def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    # sacar las n publicaciones más "hot"
    # Juntar en lotes de a 50 publicaciones más hot
    top_submissions = list(subreddit.hot(limit=n_hot)) 
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            # intentar calcular la cantidad de upvotes y downvotes de cada post
            try:
                # submission.score = u-d
                # submission.upvote_ratio = u/(u+d)
                ratio = submission.upvote_ratio
                # u = (u-d)*(u/(u+d))/(2u/(u+d) - 1) = u*((u-d)/(u+d))/((u-d)/(u+d)) -> 1
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                # guardar datos de cada post
                upvotes.append(ups)
                downvotes.append(ups - submission.score) # d = u - (u-d)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        # guardar votos de los post del batch como array
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url


Overwriting bench_module.py


In [ ]:
from bench_module import praw_reddit

In [ ]:
votes, post, url = praw_reddit()

In [ ]:
post[0]

['Discusión random semanal',
 'Pudú en mi u 🤠',
 'Medios de transporte en Chile',
 'siempre va a haber un hueon mas loco que tu en la calle',
 'La gente no sabe, pero gracias al hogar de Cristo muchas personas pueden recibir un desayuno digno por una pequeña contribución. El aporte de las voluntarias es invaluable y los beneficiados lo agradecen cariñosamente',
 'Cámara de comercio de Panguipullu busca evitar recital de marcianeke',
 'No voy a decir frases grandilocuentes tipo ‘se les acabó la fiesta’',
 'Cada día más lejos de Nueva Zelanda: En las últimas 24 horas hubo NUEVE homicidios',
 'Suprema dicta otra condena contra Krassnoff: exDINA ya suma cerca de 900 años de carcel',
 'Completo mojado o algo así. No sé, no soy de talca',
 'En Uruguay, Argentina y Paraguay hay revuelo, investigaciones penales y del Congreso por vuelos de avión venezolano iraní, incluso con pasajeros detenidos en Argentina con pasaportes falsos. Aquí en Chile noticia aún no tiene explicación. El avión estuvo 

In [ ]:
votes[0]

array([[117, 108],
       [473,   5],
       [283,  25],
       [ 81,   4],
       [109,  21],
       [265,  14],
       [ 83,  16],
       [186,  23],
       [ 55,   2],
       [ 87,  14],
       [ 40,   7],
       [ 57,   4],
       [ 56,  15],
       [ 58,   2],
       [ 67,  25],
       [410,  13],
       [ 65,   4],
       [ 27,   3],
       [ 19,   7],
       [ 12,   3],
       [ 16,   6],
       [ 23,   7],
       [ 88,  11],
       [ 40,   3],
       [ 95,   5],
       [  4,   0],
       [ 25,   3],
       [ 23,   4],
       [ 82,   5],
       [ 97,   2],
       [204,   6],
       [ 78,   6],
       [ 28,   2],
       [160,   5],
       [  6,   1],
       [ 11,   6],
       [  2,   0],
       [ 98,   7],
       [ 70,   3],
       [ 69,   2],
       [ 63,   3],
       [562,  77],
       [294,  44],
       [435,  71],
       [192,  39],
       [126,   4],
       [ 45,   4],
       [ 75,   2],
       [  1,   0],
       [  1,   0]])

In [ ]:
l = len(votes) # diccionario con arrays con votos para cada lote
assert l == len(post) # diccionario con lista con titulo de posts para cada lote 
assert l == len(url) # diccionario con lista con url de posts para cada lote 

In [ ]:
#  Cargar profiler
%load_ext line_profiler
%lprun -f praw_reddit praw_reddit() # Ejecutar profielr


```
Timer unit: 1e-06 s

Total time: 15.025 s
File: /content/bench_module.py
Function: praw_reddit at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     4                                           def praw_reddit(nombre_subreddit="chile", n_hot=1000):
     5         1          3.0      3.0      0.0      reddit = praw.Reddit(
     6         1          1.0      1.0      0.0          client_id="-w2hyFINxZ8T3g",
     7         1          0.0      0.0      0.0          client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
     8         1          1.0      1.0      0.0          password="ClasesMDS7202",
     9         1          1.0      1.0      0.0          user_agent="Clases",
    10         1          0.0      0.0      0.0          username="DocenciaDataScience",
    11         1       1700.0   1700.0      0.0          check_for_async=False,
    12                                               )
    13         1         36.0     36.0      0.0      subreddit = reddit.subreddit(nombre_subreddit)
    14                                           
    15         1          1.0      1.0      0.0      votes, post, url = {}, {}, {}
    16                                               # sacar las n publicaciones más "hot"
    17                                               # Juntar en lotes de a 50 publicaciones más hot
    18         1   14935797.0 14935797.0     99.4      top_submissions = list(subreddit.hot(limit=n_hot)) 
    19        19         37.0      1.9      0.0      for it, top_n in enumerate(range(50, len(top_submissions), 50)):
    20        18        197.0     10.9      0.0          top_n_submissions = top_submissions[:top_n]
    21        18        132.0      7.3      0.0          upvotes, downvotes, url[it], post[it] = [], [], [], []
    22                                           
    23      8568       7612.0      0.9      0.1          for submission in top_n_submissions:
    24                                                       # intentar calcular la cantidad de upvotes y downvotes de cada post
    25      8550       6659.0      0.8      0.0              try:
    26                                                           # submission.score = u-d
    27                                                           # submission.upvote_ratio = u/(u+d)
    28      8550      10743.0      1.3      0.1                  ratio = submission.upvote_ratio
    29                                                           # u = (u-d)*(u/(u+d))/(2u/(u+d) - 1) = u*((u-d)/(u+d))/((u-d)/(u+d)) -> 1
    30      8550       7821.0      0.9      0.1                  ups = int(
    31                                                               round((ratio * submission.score) / (2 * ratio - 1))
    32      8550      13924.0      1.6      0.1                      if ratio != 0.5
    33       159        219.0      1.4      0.0                      else round(submission.score / 2)
    34                                                           )
    35                                                           # guardar datos de cada post
    36      8550       7835.0      0.9      0.1                  upvotes.append(ups)
    37      8550       8660.0      1.0      0.1                  downvotes.append(ups - submission.score) # d = u - (u-d)
    38      8550      10238.0      1.2      0.1                  post[it].append(submission.title)
    39      8550      11617.0      1.4      0.1                  url[it].append(submission.url)
    40                                                       except Exception as e:
    41                                                           continue
    42                                                   # guardar votos de los post del batch como array
    43        18       1764.0     98.0      0.0          votes[it] = np.array([upvotes, downvotes]).T
    44         1          1.0      1.0      0.0      return votes, post, url
    

check
0 s
```



In [ ]:
#  Cargar profiler
%load_ext memory_profiler
%mprun -f praw_reddit praw_reddit() # Ejecutar profielr



```
Filename: /content/bench_module.py

Line #    Mem usage    Increment  Occurrences   Line Contents
=============================================================
     4    246.3 MiB    246.3 MiB           1   def praw_reddit(nombre_subreddit="chile", n_hot=1000):
     5    246.3 MiB      0.0 MiB           1       reddit = praw.Reddit(
     6    246.3 MiB      0.0 MiB           1           client_id="-w2hyFINxZ8T3g",
     7    246.3 MiB      0.0 MiB           1           client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
     8    246.3 MiB      0.0 MiB           1           password="ClasesMDS7202",
     9    246.3 MiB      0.0 MiB           1           user_agent="Clases",
    10    246.3 MiB      0.0 MiB           1           username="DocenciaDataScience",
    11    246.3 MiB      0.0 MiB           1           check_for_async=False,
    12                                             )
    13    246.3 MiB      0.0 MiB           1       subreddit = reddit.subreddit(nombre_subreddit)
    14                                         
    15    246.3 MiB      0.0 MiB           1       votes, post, url = {}, {}, {}
    16                                             # sacar las n publicaciones más "hot"
    17                                             # Juntar en lotes de a 50 publicaciones más hot
    18    263.2 MiB     17.0 MiB           1       top_submissions = list(subreddit.hot(limit=n_hot)) 
    19    263.2 MiB      0.0 MiB          19       for it, top_n in enumerate(range(50, len(top_submissions), 50)):
    20    263.2 MiB      0.0 MiB          18           top_n_submissions = top_submissions[:top_n]
    21    263.2 MiB      0.0 MiB          18           upvotes, downvotes, url[it], post[it] = [], [], [], []
    22                                         
    23    263.2 MiB      0.0 MiB        8568           for submission in top_n_submissions:
    24                                                     # intentar calcular la cantidad de upvotes y downvotes de cada post
    25    263.2 MiB      0.0 MiB        8550               try:
    26                                                         # submission.score = u-d
    27                                                         # submission.upvote_ratio = u/(u+d)
    28    263.2 MiB      0.0 MiB        8550                   ratio = submission.upvote_ratio
    29                                                         # u = (u-d)*(u/(u+d))/(2u/(u+d) - 1) = u*((u-d)/(u+d))/((u-d)/(u+d)) -> 1
    30    263.2 MiB      0.0 MiB        8550                   ups = int(
    31                                                             round((ratio * submission.score) / (2 * ratio - 1))
    32    263.2 MiB      0.0 MiB        8550                       if ratio != 0.5
    33    263.2 MiB      0.0 MiB         173                       else round(submission.score / 2)
    34                                                         )
    35                                                         # guardar datos de cada post
    36    263.2 MiB      0.0 MiB        8550                   upvotes.append(ups)
    37    263.2 MiB      0.0 MiB        8550                   downvotes.append(ups - submission.score) # d = u - (u-d)
    38    263.2 MiB      0.0 MiB        8550                   post[it].append(submission.title)
    39    263.2 MiB      0.0 MiB        8550                   url[it].append(submission.url)
    40                                                     except Exception as e:
    41                                                         continue
    42                                                 # guardar votos de los post del batch como array
    43    263.2 MiB      0.0 MiB          18           votes[it] = np.array([upvotes, downvotes]).T
    44    263.2 MiB      0.0 MiB           1       return votes, post, url
```



**Respuesta:**

Como se puede observar del profiler, la linea donde se gasta más tiempo es la linea 15 donde se cargan las n publicaciones más hot, esto tiene sentido debido a que en escencia se tienen que ordenar las publicaciones y cargar las n más votadas. 

Seguido a esto, la línea que ocupa el 2do lugar en tiempo es la linea 29, ya que corresponde a una operación if de desigualdad dentro de un loop, que se repite 9500 veces, aun así este tiempo es infimo al compararlo con el primer lugar, pero dentro de las operaciones del for loop esta es la que consume más tiempo. 

Las líneas con menos consumo de tiempo son las lineas donde se definen variables como la línea 15 o la línea 5, ya que no realiza alguna operación mas que asignar un espacio en la memoria. 


Con respecto al uso de memoria, como era de esperarse solo incrementa de forma notable en la línea 18, donde se cargan los top n posts. 








## 1.2 LRU y Análisis de Tiempo con Cache [1 Punto]

Respondan las siguientes preguntas:

1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [ ] Responder las preguntas.
- [ ] Mejorar el código con cache.
- [ ] Explicar las consecuencias de usar lru_cache sobre praw_reddit

**Respuestas Teóricas:**

`Escriba su respuesta aquí`

1. La memoria cache es una memoria generalmente más pequeña que se utiliza para mejorar el rendimiento de aplicaciones por medio de un uso eficiente de la memoria, la ideas es guardar los resultados de operaciones intensivas en la memoria para luego volver a utilizarlas o guardar datos que son accedidos frecuentemente para no perder tiempo buscándolos, este espacio puede ser de acceso rapido (RAM), disco (SSD, HDD) o almacenada de manera remota. 

LRU es sigla de Least Recently Used (menos usado recientemente), este algoritmo organiza una pequeña memoria (el cache) según el orden de uso de sus elementos, entonces se puede acceder muy rápidamente a aquellos elementos que no han sido usados, cuando el cache se llena se descarta primero el item menos usado y se añade el nuevo elemento al final de la cola para "salir" del cache. 

2. El principal costo del caching además del uso de memoria extra es que se debe seguir una política en el cache para mantener aquellos elementos que efectivamente optimizen los tiempos de ejecución. El problema es que aplicar una política (e.g: LRU) puede ser costoso computacionalmente incluso muchas veces tan sólo se realizan aproximaciones, teniendo que hacerse un trade-off entre el tiempo ahorrado de ejecución por hacer caching y el tiempo que se gasta para administrarlo.

3. Al ocupar caching en la función se guardan los datos de los posts en el cache, lo que hace más rápido su almacenamiento y posterior búsqueda. Como se vio en la sección pasada, se cargan y leen los datos de los n post más hot, por lo que al almacenar estos datos en cache incrementaria la velocidad de escritura y lectura, y por ende reduciría el tiempo total de la función.

In [ ]:
@lru_cache()
def praw_reddit_cache(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    # sacar las n publicaciones más "hot"
    # Juntar en lotes de a 50 publicaciones más hot
    top_submissions = list(subreddit.hot(limit=n_hot)) 
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            # intentar calcular la cantidad de upvotes y downvotes de cada post
            try:
                # submission.score = u-d
                # submission.upvote_ratio = u/(u+d)
                ratio = submission.upvote_ratio
                # u = (u-d)*(u/(u+d))/(2u/(u+d) - 1) = u*((u-d)/(u+d))/((u-d)/(u+d)) -> 1
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                # guardar datos de cada post
                upvotes.append(ups)
                downvotes.append(ups - submission.score) # d = u - (u-d)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        # guardar votos de los post del batch como array
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

In [ ]:
%timeit praw_reddit()

13.4 s ± 419 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit praw_reddit_cache()

The slowest run took 19.62 times longer than the fastest. This could mean that an intermediate result is being cached.
723 ns ± 1.24 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Respuesta de lru_cache sobre praw_reddit**

Como se puede ver, al utilizar lru_cache sobre praw_reddit se obtuvo una mejor considreable en el tiempo de ejecución, siendo esta del orden de 10e6 veces más rápida, ya que con lru_cache tomo 723 microsegundos y sin lru_cache tomó 13.4 segundos.



## 1.3 Obtención de Mean Posterior y Standard Error [1 Punto]

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia estadística. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$

$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [ ]:
###### Código Aquí ######
import math

def mean_std(x):
  mean = np.zeros(len(x))
  std = np.zeros(len(x))
  for i in range(len(x)):
    u, d = x[i, :]
    a = 1 + u
    b = 1 + d
    T = (a+b)
    std[i] = 1.65*math.sqrt((a*b)/(T+1))/T
    mean[i] = a/T

  return mean, std


def mean_std_np(x):
    a= x[:,0] + 1
    b= x[:,1] + 1
    T = (a+b)
    std = 1.65 * np.sqrt((a*b)/(T+1))/T
    mean = a/T

    return mean,std

mean, std = mean_std(votes[0])
mean = np.array(mean)
std = np.array(std)

mean_np, std_np = mean_std_np(votes[0])

assert np.allclose(mean_np, mean)
assert np.allclose(std_np, std)

## 1.4 Comparación de Rendimiento [2 Puntos]

Ahora, aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. 

Para esto genere un gráfico de linea utilizando plotly, en donde se pueda observar el tiempo que toma ejecutar cada función (el eje y) diferentes cantidades de datos (ejemplo, en el eje x considerar 100, 200, 300, ..., 1000 datos) sobre cada función implementada (series `Python`, `Numpy`, `Python-JIT`, `Numpy-JIT`). ¿Es posible observar diferencias? ¿A qué se debe esto?.

**Nota:** Ejecuten las funciones compiladas con algún ejemplo antes de graficar. De lo contrario, les indicará el tiempo de compilación, cosa que no queremos medir

**Nota 2**: En el caso que su computador sea muy rápido, se recomienda graficar el eje y (tiempo) como logaritmo usando `log_y=True`.


In [ ]:
@jit(nopython=True)
def mean_std_numba(x):
  mean = np.zeros(len(x))
  std = np.zeros(len(x))
  for i in range(len(x)):
    u, d = x[i, :]
    a = 1 + u
    b = 1 + d
    T = (a+b)
    std[i] = 1.65*math.sqrt((a*b)/(T+1))/T
    mean[i] = a/T

  return mean, std

@jit(nopython=True)
def mean_std_np_numba(x):
    a= x[:,0] + 1
    b= x[:,1] + 1
    T = (a+b)
    std = 1.65 * np.sqrt((a*b)/(T+1))/T
    mean = a/T

    return mean,std

# Compilar y testear
mean, std = mean_std_numba(votes[0])
mean = np.array(mean)
std = np.array(std)

mean_np, std_np = mean_std_np_numba(votes[0])

assert np.allclose(mean_np, mean)
assert np.allclose(std_np, std)

In [ ]:
# Tamaños de arrays a probar
test_sizes= np.arange(50, 1001, 50)
# Lista de funciones a probar
to_test_func = [mean_std, mean_std_np, mean_std_numba, mean_std_np_numba] 

# Para guardar tiempos promedios y stds
mean_times = np.zeros((len(to_test_func) , len(test_sizes)), dtype = np.float32)
std_times = np.zeros((len(to_test_func) , len(test_sizes)), dtype = np.float32)

# Para cada tamaño
for i,sz in enumerate(test_sizes):
  np.random.seed(test_sizes)
  # Crear array aleatorio de enteros de tamaño sz
  test_array = np.random.randint(0, high = 101, size = (sz, 2))
  # Para cada func medir tiempo de 5 repeticiones con timeit
  for j, func in enumerate(to_test_func):
    timeit_time = %timeit -q -o -r 5 func(test_array)
    mean_times[j,i] = timeit_time.average
    std_times[j,i] = timeit_time.stdev


In [ ]:
import plotly.graph_objects as go

Names = ['Python', 'Numpy', 'Python-JIT', 'Numpy-JIT']

scatters = [go.Scatter(
        name=name,
        x=test_sizes,
        y=mean_times[i],
        error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=std_times[i],
            visible=True),
        mode='markers+lines') for i,name in enumerate(Names)]

fig = go.Figure(scatters)
fig.update_yaxes(type="log")
fig

Como se puede ver en el gráfico anterior, por lejos la funcion en python nativo es la más lenta de todas. por otro lado se encuentra Numpy que es la degunda más lenta hasta al rededor de 400 en testsize donde se cruza con la curva de python-JIT, donde tienen un desempeño similar siendo python-JIT más lenta. La función más rápida es la función implementada en numpy-JIT. 

Esto ocurre en primera instancia porque ocupar numpy es más eficiente que python nativo debido a como están construidas las operaciones en la librería, y además al agregar el compilador JIT se reducen los tiempos gracias al compilado, pero se mantiene el orden en que python-JIT es más lento que numpy-JIT.

## 1.5 Plot de Resultados de Análisis de Votos [1 Punto]

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [ ]:
votos = votes[len(votes) - 1]
posteo = post[len(post) - 1]
print("lower bounds aproximados:")
posterior_mean, std_err = mean_std_np_numba(votos)
lb = posterior_mean - std_err
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {
    'Votos (+)': votos[order[:20], 0],
    'Votos (-)': votos[order[:20], 1],
    'Post': np.array(posteo)[order[:20]],
    'url': np.array(url[len(url) - 1])[order[:20]]
}
df = pd.DataFrame(data=vote_post)
ordered_post = df.Post
df

lower bounds aproximados:

Top 20 post ordenador por el limite inferior:



,Votos (+),Votos (-),Post,url
0,237,0,Gobierno queda fuera de querella por asesinato...,https://www.eldinamo.cl/pais/Gobierno-queda-fu...
1,226,0,Empresa le paga por error sueldo de $ 165 mill...,https://www.t13.cl/noticia/diario-financiero/n...
2,212,0,Macabro hallazgo en San José de Maipo: perro l...,https://www.biobiochile.cl/noticias/nacional/r...
3,1038,10,fonasa,https://i.redd.it/az7ge5c891691.png
4,708,7,ahora no po',https://i.redd.it/83odxdkaq9791.png
5,776,8,Bolos,https://i.redd.it/8tx19xd0k2691.jpg
6,694,7,hoy en fotos que envejecieron mal....,https://i.redd.it/t09uq869jg591.jpg
7,525,5,la situación de los pudúes en Chiloé es crític...,https://www.reddit.com/gallery/vhblmb
8,514,5,free market,https://i.redd.it/rt9a9kp7wt591.png
9,431,4,Otros tiempos,https://i.redd.it/3dgqvverce791.jpg


In [ ]:
df['sum_votos'] = df['Votos (+)']+df['Votos (-)']
df['rating+'] = (df['Votos (+)']/df['sum_votos']).round(2)
df['rating-'] = (df['Votos (-)']/df['sum_votos']).round(2)
df

,Votos (+),Votos (-),Post,url,sum_votos,rating+,rating-
0,237,0,Gobierno queda fuera de querella por asesinato...,https://www.eldinamo.cl/pais/Gobierno-queda-fu...,237,1.00,0.00
1,226,0,Empresa le paga por error sueldo de $ 165 mill...,https://www.t13.cl/noticia/diario-financiero/n...,226,1.00,0.00
2,212,0,Macabro hallazgo en San José de Maipo: perro l...,https://www.biobiochile.cl/noticias/nacional/r...,212,1.00,0.00
3,1038,10,fonasa,https://i.redd.it/az7ge5c891691.png,1048,0.99,0.01
4,708,7,ahora no po',https://i.redd.it/83odxdkaq9791.png,715,0.99,0.01
5,776,8,Bolos,https://i.redd.it/8tx19xd0k2691.jpg,784,0.99,0.01
6,694,7,hoy en fotos que envejecieron mal....,https://i.redd.it/t09uq869jg591.jpg,701,0.99,0.01
7,525,5,la situación de los pudúes en Chiloé es crític...,https://www.reddit.com/gallery/vhblmb,530,0.99,0.01
8,514,5,free market,https://i.redd.it/rt9a9kp7wt591.png,519,0.99,0.01
9,431,4,Otros tiempos,https://i.redd.it/3dgqvverce791.jpg,435,0.99,0.01


In [ ]:
r_order = order[::-1][-20:]
data_dic = {
    'mean': posterior_mean[r_order],
    'std_err': std_err[r_order],
    'post': ordered_post[::-1]
}
df = pd.DataFrame(data=data_dic)
fig = px.scatter(df, x="mean", y="post", error_x="std_err")
fig.show()

**Respuesta:**

Es relativamente intuitivo que cuando hay muchos votos utilizar (a/(a+b)) sirve como proxy para confiar que el post es bueno, pero cuando se reduce la cantidad de votos esta medida se vuelve engañosa (por ejemplo el último post) y la desviación del resultado se vuelve lo suficientemente grande por lo que ya no se puede confiar en el score del post, esto explica poruqe solo hay post con una cantidad relativamente alta de votos. Por otro lado consideramos poco intuitivo lo mucho que afectan las primeras reviews negativas a la confianza de un score, por ejemplo se observan post con alrededor de 200-250 reviews con 0 negativos que estan posicionados bastante por encima en terminos de lb que unos con una cantidad de votos cercana pero con 2 reviews negativas (1%).



<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>